In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
import tqdm

In [2]:
EPOCH = 15
BATCH_SIZE = 128
LR = 0.001
DOWNLOAD_MNIST = True

In [3]:
train_data = torchvision.datasets.MNIST(root='./mnist',
                                       train=True,
                                       transform=torchvision.transforms.ToTensor(),
                                       download=DOWNLOAD_MNIST)

In [4]:
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist', train=False)


In [5]:
test_x = Variable(torch.unsqueeze(test_data.test_data, 1), 
                  volatile=True).type(torch.FloatTensor).cuda() / 255
test_y = test_data.test_labels.cuda()

C:\Users\zdwxx\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\zdwxx\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\zdwxx\AppData\Local\Programs\Python\Python37\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [6]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1,
                padding=2,
            ),# (16,28,28)
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2,
            ),  # (16,14,14)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=16,
                out_channels=32,
                kernel_size=5,
                stride=1,
                padding=2,
            ), #(32*14*14)
            nn.ReLU(),
            nn.MaxPool2d(
                kernel_size=2,
            ), # (32*7*7)
        )
        self.out = nn.Linear(32*7*7, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

In [7]:
cnn = CNN()
cnn.cuda()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [8]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()

In [9]:
for epoch in range(EPOCH):
    for step, (x, y) in tqdm.tqdm(list(enumerate(train_loader))):
        b_x = Variable(x).cuda()
        b_y = Variable(y).cuda()
        
        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print("loss:", loss.data.cpu().numpy())

100%|████████████████████████████████████████████████████████████████████████████████| 469/469 [00:05<00:00, 90.24it/s]


loss: 0.09811235


100%|███████████████████████████████████████████████████████████████████████████████| 469/469 [00:04<00:00, 109.45it/s]


loss: 0.017910631


100%|███████████████████████████████████████████████████████████████████████████████| 469/469 [00:04<00:00, 116.31it/s]


...

100%|███████████████████████████████████████████████████████████████████████████████| 469/469 [00:04<00:00, 114.93it/s]


loss: 0.017410308


100%|███████████████████████████████████████████████████████████████████████████████| 469/469 [00:04<00:00, 113.70it/s]


loss: 0.0028187137


KeyboardInterrupt: 

In [10]:
test_output = cnn(test_x)
pred_y = torch.argmax(test_output, 1).data.cpu().numpy()

In [11]:
real = test_y.cpu().numpy()

In [12]:
sum(pred_y == real) / test_output.size(0)

0.9896